In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
from report_utils import excel_report

import numpy as np
from log import log

In [2]:
# проверка телефона на валидность

def valid_phone(phone):
    try:
        if len(phone) == 11:
            return phone
        else:
            return 'Invalid phone'
    except:
        return 'Invalid phone'

In [3]:
# получение всех пациентов

sql_patients = """
select p.id, 
	   p.person_id,
	   per.phone,
	   --per.partner_id,
	   --par."name", 
	   case 
        	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
        	else 'ПРОЧИЕ' 
       end "name",
	   case
	   		when (((substring(per.phone, 2, 3) = '901' and cast(substring(per.phone, 5, 7) as numeric) between 9460000 and 9469999) 
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7110000 and 7119999)
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7150000 and 7159999)
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 0610000 and 0629999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3050000 and 3079999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3130000 and 3149999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3180000 and 3199999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3400000 and 3449999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3870000 and 3889999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 6600000 and 6699999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 6700000 and 6799999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7120000 and 7179999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7600000 and 7699999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 0200000 and 0269999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 0380000 and 0399999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 3100000 and 3199999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 3700000 and 3799999)
	   			 or (substring(per.phone, 2, 3) = '906' and cast(substring(per.phone, 5, 7) as numeric) between 1100000 and 1249999)
	   			 or (substring(per.phone, 2, 3) = '906' and cast(substring(per.phone, 5, 7) as numeric) between 3200000 and 3339999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3300000 and 3399999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3400000 and 3439999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3460000 and 3499999)
	   			 or (substring(per.phone, 2, 3) = '909' and cast(substring(per.phone, 5, 7) as numeric) between 3060000 and 3149999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 2200000 and 2999999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 3900000 and 3999999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 8500000 and 9399999)
	   			 or (substring(per.phone, 2, 3) = '919' and cast(substring(per.phone, 5, 7) as numeric) between 6200000 and 6499999)
	   			 or (substring(per.phone, 2, 3) = '919' and cast(substring(per.phone, 5, 7) as numeric) between 6800000 and 6999999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 0300000 and 0399999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 0400000 and 0499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 2400000 and 2499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 4000000 and 4499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 4500000 and 4999999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 6700000 and 6799999)
	   			 or (substring(per.phone, 2, 3) = '937' and cast(substring(per.phone, 5, 7) as numeric) between 2900000 and 2999999)
	   			 or (substring(per.phone, 2, 3) = '950' and cast(substring(per.phone, 5, 7) as numeric) between 3100000 and 3299999)
	   			 or (substring(per.phone, 2, 3) = '950' and cast(substring(per.phone, 5, 7) as numeric) between 6650000 and 6689999)
	   			 or (substring(per.phone, 2, 3) = '951' and cast(substring(per.phone, 5, 7) as numeric) between 0600000 and 0699999)
	   			 or (substring(per.phone, 2, 3) = '951' and cast(substring(per.phone, 5, 7) as numeric) between 8900000 and 8999999)
	   			 or (substring(per.phone, 2, 3) = '960' and cast(substring(per.phone, 5, 7) as numeric) between 0300000 and 0899999)
	   			 or (substring(per.phone, 2, 3) = '962' and cast(substring(per.phone, 5, 7) as numeric) between 5480000 and 5799999)
	   			 or (substring(per.phone, 2, 3) = '963' and cast(substring(per.phone, 5, 7) as numeric) between 1200000 and 1259999)
	   			 or (substring(per.phone, 2, 3) = '987' and cast(substring(per.phone, 5, 7) as numeric) between 2900000 and 2999999)
	   			) and length(phone) = 11
	   			)
	   		then 'Республика Татарстан'
	   		else 'Другой регион'
	   end region
from patient p 
left join person per on p.person_id = per.id 
left join patient_promotion pp on p.id = pp.patient_id 
left join promotion promo on pp.promotion_id = promo.id
left join partner par on promo.partner_id = par.id
--where par."name" = 'ВЭБ'
"""

In [4]:
# олучение всех консультаций

sql_app = """
select a.id appoint_id,
	   p.person_id,
	   per.phone,
	   --per.partner_id,
	   --par."name", 
	   case 
        	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
        	else 'ПРОЧИЕ' 
       end "name",
	   case
	   		when (((substring(per.phone, 2, 3) = '901' and cast(substring(per.phone, 5, 7) as numeric) between 9460000 and 9469999) 
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7110000 and 7119999)
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7150000 and 7159999)
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 0610000 and 0629999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3050000 and 3079999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3130000 and 3149999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3180000 and 3199999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3400000 and 3449999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3870000 and 3889999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 6600000 and 6699999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 6700000 and 6799999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7120000 and 7179999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7600000 and 7699999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 0200000 and 0269999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 0380000 and 0399999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 3100000 and 3199999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 3700000 and 3799999)
	   			 or (substring(per.phone, 2, 3) = '906' and cast(substring(per.phone, 5, 7) as numeric) between 1100000 and 1249999)
	   			 or (substring(per.phone, 2, 3) = '906' and cast(substring(per.phone, 5, 7) as numeric) between 3200000 and 3339999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3300000 and 3399999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3400000 and 3439999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3460000 and 3499999)
	   			 or (substring(per.phone, 2, 3) = '909' and cast(substring(per.phone, 5, 7) as numeric) between 3060000 and 3149999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 2200000 and 2999999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 3900000 and 3999999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 8500000 and 9399999)
	   			 or (substring(per.phone, 2, 3) = '919' and cast(substring(per.phone, 5, 7) as numeric) between 6200000 and 6499999)
	   			 or (substring(per.phone, 2, 3) = '919' and cast(substring(per.phone, 5, 7) as numeric) between 6800000 and 6999999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 0300000 and 0399999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 0400000 and 0499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 2400000 and 2499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 4000000 and 4499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 4500000 and 4999999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 6700000 and 6799999)
	   			 or (substring(per.phone, 2, 3) = '937' and cast(substring(per.phone, 5, 7) as numeric) between 2900000 and 2999999)
	   			 or (substring(per.phone, 2, 3) = '950' and cast(substring(per.phone, 5, 7) as numeric) between 3100000 and 3299999)
	   			 or (substring(per.phone, 2, 3) = '950' and cast(substring(per.phone, 5, 7) as numeric) between 6650000 and 6689999)
	   			 or (substring(per.phone, 2, 3) = '951' and cast(substring(per.phone, 5, 7) as numeric) between 0600000 and 0699999)
	   			 or (substring(per.phone, 2, 3) = '951' and cast(substring(per.phone, 5, 7) as numeric) between 8900000 and 8999999)
	   			 or (substring(per.phone, 2, 3) = '960' and cast(substring(per.phone, 5, 7) as numeric) between 0300000 and 0899999)
	   			 or (substring(per.phone, 2, 3) = '962' and cast(substring(per.phone, 5, 7) as numeric) between 5480000 and 5799999)
	   			 or (substring(per.phone, 2, 3) = '963' and cast(substring(per.phone, 5, 7) as numeric) between 1200000 and 1259999)
	   			 or (substring(per.phone, 2, 3) = '987' and cast(substring(per.phone, 5, 7) as numeric) between 2900000 and 2999999)
	   			) and length(phone) = 11
	   			)
	   		then 'Республика Татарстан'
	   		else 'Другой регион'
	   end region
from appointment a
left join patient p on a.patient_id = p.id 
left join person per on p.person_id = per.id 
left join patient_promotion pp on p.id = pp.patient_id 
left join promotion promo on pp.promotion_id = promo.id
left join partner par on promo.partner_id = par.id
"""

In [5]:
connection = UniversalConnection('../../.credentials/telemed/prom.cfg', DBType.Postgres)

2020-04-08 12:39:14 connect to postgres database using config file "../../.credentials/telemed/prom.cfg"
2020-04-08 12:39:15 creating ssh tunnel to 172.16.100.19 as root...
2020-04-08 12:39:25 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 42957
2020-04-08 12:39:25 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [6]:
data_pat = connection.query(sql_patients)

2020-04-08 12:39:25 @telemed query:
                    select p.id, 
				    	   p.person_id,
				    	   per.phone,
				    	   --per.partner_id,
				    	   --par."name", 
				    	   case 
				            	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
				            	else 'ПРОЧИЕ' 
				           end "name",
				    	   case
				    	   		when (((substring(per.phone, 2, 3) = '901' and cast(substring(per.phone, 5, 7) as numeric) between 9460000 and 9469999) 
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7110000 and 7119999)
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7150000 and 7159999)
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
				    	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 06100

In [7]:
data_app = connection.query(sql_app)

2020-04-08 12:40:49 @telemed query:
                    select a.id appoint_id,
				    	   p.person_id,
				    	   per.phone,
				    	   --per.partner_id,
				    	   --par."name", 
				    	   case 
				            	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
				            	else 'ПРОЧИЕ' 
				           end "name",
				    	   case
				    	   		when (((substring(per.phone, 2, 3) = '901' and cast(substring(per.phone, 5, 7) as numeric) between 9460000 and 9469999) 
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7110000 and 7119999)
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7150000 and 7159999)
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
				    	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) bet

In [8]:
connection.close()

2020-04-08 12:40:54 @telemed: closing postgres connection...
2020-04-08 12:40:54 closing ssh tunnel to 172.16.100.19...


In [9]:
df = pd.DataFrame(data_pat)

In [10]:
df['phone'] = df['phone'].apply(valid_phone)
# замена невалидных телефонов на "Invalid phone"

total_patients = df.shape[0]
# подсчет общего количества записей в таблице

patients_with_valid_phone = total_patients - df[df['phone'] == 'Invalid phone'].shape[0]
# подсчет пациентов с валидным телефоном

patients_web = df[(df['phone'] != 'Invalid phone') & 
                  (df['name'] == 'VEBMED')].shape[0]
# подсчет пациентов от ВЭБа с валидным телефоном

patient_tatarstan = df[(df['phone'] != 'Invalid phone') &
                       (df['region'] == 'Республика Татарстан')].shape[0]
# подсчет пациентов из Татарстана

patient_tatarstan_web = df[(df['phone'] != 'Invalid phone') & 
                           (df['region'] == 'Республика Татарстан') & 
                           (df['name'] == 'VEBMED')].shape[0]
# подсчет пациентов ВЭБа из Татарстана

In [11]:
df_app = pd.DataFrame(data_app)

In [12]:
# тут все аналогично пациентам
df_app['phone'] = df_app['phone'].apply(valid_phone)

total_appointment = df_app.shape[0]

appointment_with_valid_phone = total_appointment - df_app[df_app['phone'] == 'Invalid phone'].shape[0]

appointment_web = df_app[(df_app['phone'] != 'Invalid phone') & 
                         (df_app['name'] == 'VEBMED')].shape[0]

appointment_tatarstan = df_app[(df_app['phone'] != 'Invalid phone') & 
                               (df_app['region'] == 'Республика Татарстан')].shape[0]

appointment_tatarstan_web = df_app[(df_app['phone'] != 'Invalid phone') & 
                                   (df_app['region'] == 'Республика Татарстан') & 
                                   (df_app['name'] == 'VEBMED')].shape[0]

In [13]:
final_report = {
          'Всего': [total_patients, total_appointment],
          'С телефоном': [patients_with_valid_phone, appointment_with_valid_phone],
          'По VEBMED': [patients_web, appointment_web],
          'Из Татарстана': [patient_tatarstan, appointment_tatarstan],
          'По VEBMED из Татарстана': [patient_tatarstan_web, appointment_tatarstan_web]   
}

In [14]:
report = pd.DataFrame.from_dict(final_report, orient='index')

In [15]:
report.columns = ['Пациентов', 'Консультаций']

In [16]:
report

,Пациентов,Консультаций
Всего,4886494,431958
С телефоном,4861286,431770
По VEBMED,9274,18093
Из Татарстана,80588,6602
По VEBMED из Татарстана,448,440


In [17]:
report.to_csv('Report_Tatarstan.csv')